# Lab 1: Review and IVPs

**Gwen Lofman**, ISC4232C

In [ ]:
(require '[clojupyter.misc.helper :as helper])
(require '[clojupyter.misc.display :as display])

(helper/add-dependencies '[incanter "1.5.7"])
(helper/add-dependencies '[org.clojure/math.numeric-tower "0.0.4"])

(require '[clojure.core.reducers :as r])
(require '[clojure.math.numeric-tower :as math])
(use '(incanter core stats charts io))

## Newton's Method

Newton's method iteratively solves for the roots of a funciton by "chasing" the functions derivative.  It is a well known method for it's good perfofrmance, but it may fail to converge in certain circumstances, so the choice of the initial position $x_0$ to search through the domain can change the solution.

In [ ]:
(defn newtons_method
  "Newton's method for finding the roots of a function

  Use the function `f` and its derivative `df` to calculate its
  roots starting from the initial guess `x0`.  Newton's method
  is sensitive to the initial guess used for `x0`; in other words,
  it may converge with one choice for `x0` but not for another.

  To prevent infinite recursion in the case that Newton's method
  doesn't converge, terminate early after `maxit` iterations.
  Otherwise, terminate once the difference between successive
  iterations is less than `tol`.

  Returns the solution and the number of iterations it took to
  reach it ([xk i])."
  [maxit tol f df x0]
  (loop [i 1 x x0]
    (let [xk (- x (/ (f x) (df x)))]
      (if (or (>= i maxit) (> tol (Math/abs ^double (- xk x))))
        [(double xk) i]
        (recur (inc i) xk)))))

To check that our function operates as expected, use the test function $f(x) = x^2 - 1$ because $f(x) = 0$ when $x = -1,1$.

In [ ]:
(let [f  #(- (* % %) 1) ;; Test function
      df #(* 2 %)       ;; Derivative
      x0 2]             ;; Initial Point
  (display/latex
   (str
    "\\begin{array}{l l l l}"
    (reduce (fn [s k]
              (let [[x_k i] (newtons_method k 1.0E-8 f df x0)]
                (str s (clojure.pprint/cl-format
                        nil " ~d & ~8e & ~12e & ~8e \\\\\n"
                        k x_k (Math/abs (- x_k 1)) (f x_k)))))
            "k & x_k & abs(x_k - 1) & f(x_k) \\\\\\hline\n"
            (range 1 7))
    "\\end{array}")))

This indicates that the function works as expected: it demonstrates that as the number of iterations ($k$) increases, we converge on the solution pretty quickly.

## Langrange Interpolation

Lagrange interpolation defines a polynomial which approximates a given function $f$ from a set of points $\{x_i\}^n_{i=1}$.  Here we examine the performance of Langrange Interpolation in terms of the error between the function $f$ and its interpolating function.

In [ ]:
(defn lagrange_basis
  "Basis set of Lagrange Polynomials evaluated at x

  Given a set of interpolating points `xis` and the target point
  `x`, return a vector of each Lagrange Polynomial evaluated at
  the target point."
  [xis x]
  (letfn [;; Evaluate value of L_i(x)
          (lagrange [x xi xs]
            (transduce (map #(/ (- x %) (- xi %))) * xs))
          ;; A set xs without the point x
          (except [x xs]
            (into [] (clojure.set/difference (set xs) #{x})))]
    (mapv #(lagrange x % (except % xis)) xis)))

In [ ]:
(defn lagrange_interp
  "Interpolate a function at x from points on the function

  Given a vector of points `ps` in the form `[[x y]]`, use
  Lagrange interpolation to approximate the value of the
  function at the point `x` and return the estimate `y`."
  [ps x]
  (let [ys (mapv second ps)
        xs (lagrange_basis (mapv first ps) x)]
    (reduce + (map #(* %1 %2) xs ys))))

In [ ]:
(defn lagrange-points
  "Calculate points for interpolation

  Use the function `f` and a set `xs` of points to interpolate"
  [f xs] (mapv #(vector % (f %)) xs))

### Runge's Phenomenon

To verify the interpolation code's implementations, we investivate Runge's phenomenon where the quality of the interpolation degrades at the edge of the interpolated region.

Using lagrange interpolation on Runge's function we can see this phenomenon in action.

$$\text{runge}(x) = \frac{1}{25x^2 + 1}$$

In [ ]:
(defn runge [x]
  (/ 1 (+ 1 (* 25 (* x x)))))

(-> (function-plot runge -1 1 :title "Runge's function")
    (.createBufferedImage 600 400))


We compare, for $n$ interpolating points, the visual accuracy of using uniformly selected points versus chebyshev points.

In [ ]:
(defn abs-err [v a]
  (Math/abs (- v a)))

(defn chebyshev [n]
  (mapv #(Math/cos (* Math/PI (/ ( - (* 2 %) 1) (* 2 n))))
        (range 1 (inc n))))

(defn uniform [n]
  (range -1 1.0001 (/ 2.000001 (dec n))))

(defn runges-phenomenon [f n & basis]
  (reduce
   (fn [plot {:keys [name basis]}]
     (let [ps (lagrange-points #(f %) (basis n))
           plot-name (str name " interpolation")]
       (-> plot
           (add-points (mapv first ps) (mapv second ps)
                       :series-label name)
           (add-function #(lagrange_interp ps %) -1 1
                         :series-label plot-name))))
   (function-plot #(f %) -1 1
                  :title (str n " Point Interpolation")
                  :y-label "f(x)"
                  :legend true
                  :series-label "f(x)")
   basis))

(defn runges-error [f n]
  (let [ups (lagrange-points #(f %) (uniform n))
        cps (lagrange-points #(f %) (chebyshev n))
        err (fn [ps]
              #(abs-err (f %) (lagrange_interp ps %)))]
    (-> (function-plot (err ups) -1 1
                       :title "Error"
                       :legend true
                       :series-label "uniform error")
        (add-function (err cps) -1 1
                      :series-label "chebyshev error")
        (.createBufferedImage 600 200))))


#### 3 Points

In [ ]:
(-> (runges-phenomenon runge 3 {:name "uniform" :basis uniform} {:name "chebyshev" :basis chebyshev})
    (.createBufferedImage 600 400))

In [ ]:
(runges-error runge 3)

#### 5 Points

In [ ]:
(-> (runges-phenomenon runge 5 {:name "uniform" :basis uniform} {:name "chebyshev" :basis chebyshev})
    (.createBufferedImage 600 400))

In [ ]:
(runges-error runge 5)

#### 9 Points

In [ ]:
(-> (runges-phenomenon runge 9 {:name "uniform" :basis uniform} {:name "chebyshev" :basis chebyshev})
    (.createBufferedImage 600 400))

In [ ]:
(runges-error runge 9)

#### 17 Points

In [ ]:
(-> (runges-phenomenon runge 17 {:name "uniform" :basis uniform})
    (.createBufferedImage 600 400))

In [ ]:
(-> (runges-phenomenon runge 17 {:name "chebyshev" :basis chebyshev})
    (.createBufferedImage 600 400))

In [ ]:
(runges-error runge 17)

### Runge's Phenomenon Cont.

We can perform a similar examination of the function $f(x) = \vert x \vert$.

In [ ]:
(-> (runges-phenomenon #(Math/abs %) 3 {:name "uniform" :basis uniform} {:name "chebyshev" :basis chebyshev})
    (.createBufferedImage 600 400))

In [ ]:
(runges-error #(Math/abs %) 3)

In [ ]:
(-> (runges-phenomenon #(Math/abs %) 5 {:name "uniform" :basis uniform} {:name "chebyshev" :basis chebyshev})
    (.createBufferedImage 600 400))

In [ ]:
(runges-error #(Math/abs %) 5)

In [ ]:
(-> (runges-phenomenon #(Math/abs %) 9 {:name "uniform" :basis uniform} {:name "chebyshev" :basis chebyshev})
    (.createBufferedImage 600 400))

In [ ]:
(runges-error #(Math/abs %) 9)

In [ ]:
(-> (runges-phenomenon #(Math/abs %) 17 {:name "uniform" :basis uniform})
    (.createBufferedImage 600 400))

In [ ]:
(-> (runges-phenomenon #(Math/abs %) 17 {:name "chebyshev" :basis chebyshev})
    (.createBufferedImage 600 400))

In [ ]:
(runges-error #(Math/abs %) 17)

### Analysis

Interestingly, the error when interpolating each function appears quite similar in shape across the different number of interpolating points.  This may suggest that using the chebyshev points as the basis for lagrange interpolation can reduce error no matter what function is being interpolated.  We can convince ourselves of this further by shifting runge's function by 2.

In [ ]:
(runges-phenomenon #(runge (+ % 2)) 9)

here we examine that the chebyshev points have larger error towards the center of the distribution, but as the magnitude of the function's values decreases, so does the error on the function.

In [ ]:
(runges-error #(runge (+ % 2)) 9)

## Euler Methods

Euler methods are numerical methods for solving ODEs.  In this case we will examine the initial value problem:

$$
\begin{array}{ l r }
  y'(t) = −ty(t), & y(0) = 1\\
\end{array}
$$

In [ ]:
(def y0 1)

(defn y' [t yn] (- (* t yn)))

With the exact solution:

$$
\begin{array}{ l r }
  y(t) = e^{\frac{-t^2}{2}}, & 0 \leq t \leq 1\\
\end{array}
$$

In [ ]:
(defn y [t] (math/expt Math/E (/ (- (* t t)) 2)))

(-> (function-plot y 0 1 :title "y(t)" :x-label "0 < t < 1")
    (.createBufferedImage 600 400))

In [ ]:
(defn compare-method
  "Create a plot comparing y(t) to Euler method solutions"
  [title y t0 T & series]
  (reduce (fn [plot {:keys [series ys ts]}]
            (add-lines plot ts ys :series-label series))
          (function-plot y t0 T :title title :legend true)
          series))

### Forward Euler

In [ ]:
(defn forward_euler
  "Solve the IVP for `f` with initial conditions `y0` and `t0`

  Will take steps of size `dt` until it reaches the final time
  `T`.  Error is proportional to the size of the time-step `dt`,
  but so is performance.

  Returns the vector of approximate solutions `ys` and their t
  values `ts` as [`ts` `ys`]"
  [y0 t0 T dt f]
  (loop [yn y0 tn t0 ys [] ts []]
    (if (< tn T)
      (recur (+ yn (* dt (f yn tn)))
             (+ dt tn)
             (conj ys yn)
             (conj ts tn))
      [(conj ys yn) (conj ts tn)])))

In [ ]:
(defn compare-forward
  "Compare Forward Euler for the given dts"
  [dts]
  (apply (partial compare-method "y(t) vs Forward Euler" y 0 1)
         (reduce
          #(let [s (str "dt of 1/" %2)
                 dt (double (/ 1 %2))
                 [ys ts] (forward_euler 1 0 1 dt y')]
             (conj %1 {:series s :ys ys :ts ts}))
          []
          dts)))

In [ ]:
(-> (compare-forward [2 4 8 16 32])
    (.createBufferedImage 600 400))

## Backward Euler